# Avaliacao das metricas dos modelos

Este notebook compila as principais metricas observadas durante os treinamentos recentes (RMSE, MAE, R2, MAPE e acuracias relativas) para apoiar decisoes sobre o melhor modelo a ser promovido em producao.


In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn-v0_8-whitegrid")
sns.set_palette("viridis")


In [ ]:
# Caminho relativo ao repo
root = Path(__file__).resolve().parents[2]
comparacao_path = root / "ml" / "artifacts" / "comparacao_modelos_full.csv"

if not comparacao_path.exists():
    raise FileNotFoundError(f"Arquivo nao encontrado: {comparacao_path}")

df_metricas = pd.read_csv(comparacao_path)
df_metricas


In [ ]:
def melhores_por_metrica(df: pd.DataFrame, metricas_min, metricas_max):
    registros = []
    for metrica in metricas_min:
        linha = df.loc[df[metrica].idxmin()]
        registros.append({"metrica": metrica, "modelo": linha["modelo"], "valor": linha[metrica], "criterio": "menor"})
    for metrica in metricas_max:
        linha = df.loc[df[metrica].idxmax()]
        registros.append({"metrica": metrica, "modelo": linha["modelo"], "valor": linha[metrica], "criterio": "maior"})
    return pd.DataFrame(registros)

metricas_min = ["rmse", "mae", "mape"]
metricas_max = ["r2", "accuracy_10pct", "accuracy_20pct"]
melhores_metricas = melhores_por_metrica(df_metricas, metricas_min, metricas_max)
melhores_metricas


In [ ]:
def rank_composto(df: pd.DataFrame) -> pd.DataFrame:
    normalizado = df.copy()
    for coluna in metricas_min:
        normalizado[coluna + "_score"] = 1 - (normalizado[coluna] - normalizado[coluna].min()) / (normalizado[coluna].max() - normalizado[coluna].min())
    for coluna in metricas_max:
        normalizado[coluna + "_score"] = (normalizado[coluna] - normalizado[coluna].min()) / (normalizado[coluna].max() - normalizado[coluna].min())
    score_cols = [c for c in normalizado.columns if c.endswith("_score")]
    normalizado["score_composto"] = normalizado[score_cols].mean(axis=1)
    return normalizado.sort_values("score_composto", ascending=False)[["modelo", "tipo", "score_composto"] + score_cols]

ranking = rank_composto(df_metricas)
ranking.head(5)


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

sns.barplot(data=df_metricas, x="modelo", y="r2", ax=axes[0, 0])
axes[0, 0].set_title("R2 por modelo")
axes[0, 0].tick_params(axis="x", rotation=45, ha="right")

sns.barplot(data=df_metricas, x="modelo", y="mae", ax=axes[0, 1])
axes[0, 1].set_title("MAE por modelo (quanto menor melhor)")
axes[0, 1].tick_params(axis="x", rotation=45, ha="right")

sns.barplot(data=df_metricas, x="modelo", y="rmse", ax=axes[1, 0])
axes[1, 0].set_title("RMSE por modelo (quanto menor melhor)")
axes[1, 0].tick_params(axis="x", rotation=45, ha="right")

sns.barplot(data=df_metricas, x="modelo", y="accuracy_10pct", ax=axes[1, 1])
axes[1, 1].set_title("Acuracia de +-10% por modelo")
axes[1, 1].tick_params(axis="x", rotation=45, ha="right")

plt.tight_layout()
plt.show()


## Observacoes rapidas

- Gradient Boosting e Random Forest lideram em R2, MAE e RMSE, sustentando a decisao atual de promover o Gradient Boosting.
- XGBoost baseline ainda e competitivo em acuracias relativas, sugerindo monitorar futuras versoes com features espaciais.
- Modelos lineares e redes neurais nao atingiram desempenho aceitavel; servem como referencias de controle.
